In [1]:
import cv2
import numpy as np

In [2]:
def pyramid_blending(filename_1, filename_2, height=6):
    A = cv2.imread(filename_1)
    B = cv2.imread(filename_2)

    G = A.copy()
    gpA = [G]
    for i in range(height):
        G = cv2.pyrDown(G)
        gpA.append(G)

    G = B.copy()
    gpB = [G]
    for i in range(height):
        G = cv2.pyrDown(G)
        gpB.append(G)

    lpA = [gpA[height - 1]]
    for i in range(height - 1, 0, -1):
        GE = cv2.pyrUp(gpA[i])
        rows, cols, _ = gpA[i - 1].shape
        GE = GE[:rows, :cols]
        L = cv2.subtract(gpA[i - 1], GE)
        lpA.append(L)

    lpB = [gpB[height - 1]]
    for i in range(height - 1, 0, -1):
        GE = cv2.pyrUp(gpB[i])
        rows, cols, _ = gpB[i - 1].shape
        GE = GE[:rows, :cols]
        L = cv2.subtract(gpB[i - 1], GE)
        lpB.append(L)

    LS = []
    for la, lb in zip(lpA, lpB):
        _, cols, _ = la.shape
        ls = np.hstack((la[:, :int(cols / 2)], lb[:, int(cols / 2):]))
        LS.append(ls)

    ls_ = LS[0]
    for i in range(1, height):
        ls_ = cv2.pyrUp(ls_)
        rows, cols, _ = LS[i].shape
        ls_ = ls_[:rows, :cols]
        ls_ = cv2.add(ls_, LS[i])

    real = np.hstack((A[:, :int(cols / 2)], B[:, int(cols / 2):]))

    cv2.imwrite("pyramid_blending_" + str(height) + ".jpg", ls_)
    cv2.imwrite("direct_blending.jpg", real)

In [3]:
pyramid_blending("apple.jpg", "orange.jpg", height=10)

In [4]:
def alpha_blending(foreground_filename, background_filename, alpha_mask_filename):
    foreground = cv2.imread(foreground_filename)
    background = cv2.imread(background_filename)
    alpha = cv2.imread(alpha_mask_filename)
    foreground = foreground.astype(float)
    background = background.astype(float)
    alpha = alpha.astype(float) / 255
    foreground = cv2.multiply(alpha, foreground)
    background = cv2.multiply(1.0 - alpha, background)
    res = cv2.add(foreground, background)
    cv2.imwrite("alpha_blending.jpg", res)

In [5]:
alpha_blending("foreground.png", "background.png", "mask.png")